In [ ]:
import sqlite3
from tkinter import *
from tkinter import messagebox
from tkinter import ttk
# Set up the SQLite Database
def initialize_db():
    conn = sqlite3.connect('students.db')
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS students (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            age INTEGER,
            gender TEXT,
            course TEXT
        )
    """)
    conn.commit()
    conn.close()
# Define Functions for CRUD Operations
def add_student():
    name = entry_name.get()
    age = entry_age.get()
    gender = gender_var.get()
    course = entry_course.get()
    
    if name and age and gender and course:
        conn = sqlite3.connect('students.db')
        cursor = conn.cursor()
        cursor.execute("INSERT INTO students (name, age, gender, course) VALUES (?, ?, ?, ?)",
                       (name, age, gender, course))
        conn.commit()
        conn.close()
        messagebox.showinfo("Success", "Student added successfully!")
        display_students()
    else:
        messagebox.showwarning("Input Error", "All fields are required.")
# Function to display Student in Treeview
def display_students():
    for row in tree.get_children():
        tree.delete(row)
    
    conn = sqlite3.connect('students.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM students")
    rows = cursor.fetchall()
    for row in rows:
        tree.insert("", END, values=row)
    conn.close()

# Function to update Student's data
def update_student():
    selected_item = tree.selection()
    if selected_item:
        student_id = tree.item(selected_item)['values'][0]
        name = entry_name.get()
        age = entry_age.get()
        gender = gender_var.get()
        course = entry_course.get()
        
        conn = sqlite3.connect('students.db')
        cursor = conn.cursor()
        cursor.execute("UPDATE students SET name=?, age=?, gender=?, course=? WHERE id=?",
                       (name, age, gender, course, student_id))
        conn.commit()
        conn.close()
        messagebox.showinfo("Success", "Student data updated successfully!")
        display_students()
# Function to delete student's data
def delete_student():
    selected_item = tree.selection()
    if selected_item:
        student_id = tree.item(selected_item)['values'][0]
        conn = sqlite3.connect('students.db')
        cursor = conn.cursor()
        cursor.execute("DELETE FROM students WHERE id=?", (student_id,))
        conn.commit()
        conn.close()
        messagebox.showinfo("Success", "Student deleted successfully!")
        display_students()
    else:
        messagebox.showwarning("Selection Error", "No student selected.")
# Set Up the Tkinter GUI
root = Tk()
root.title("Student Management System")

# Input fields
Label(root, text="Name").grid(row=0, column=0)
entry_name = Entry(root)
entry_name.grid(row=0, column=1)

Label(root, text="Age").grid(row=1, column=0)
entry_age = Entry(root)
entry_age.grid(row=1, column=1)

Label(root, text="Gender").grid(row=2, column=0)
gender_var = StringVar()
OptionMenu(root, gender_var, "Male", "Female", "Other").grid(row=2, column=1)

Label(root, text="Course").grid(row=3, column=0)
entry_course = Entry(root)
entry_course.grid(row=3, column=1)

# Buttons for CRUD operations
Button(root, text="Add Student", command=add_student).grid(row=4, column=0)
Button(root, text="Update Student", command=update_student).grid(row=4, column=1)
Button(root, text="Delete Student", command=delete_student).grid(row=4, column=2)

# Treeview for displaying students
tree = ttk.Treeview(root, columns=("ID", "Name", "Age", "Gender", "Course"), show="headings")
tree.heading("ID", text="ID")
tree.heading("Name", text="Name")
tree.heading("Age", text="Age")
tree.heading("Gender", text="Gender")
tree.heading("Course", text="Course")
tree.grid(row=5, column=0, columnspan=3)

# Initialize database and display data
initialize_db()
display_students()

root.mainloop()
